In [ ]:
import folium
import json
import requests
import geocoder
from IPython.display import display, HTML

In [ ]:
CLIENT_ID = "O7rCPOsuUXYVr_N-n6gSGw"
API_KEY = "mPNTc2aspvZn6I8Z2iv0-xBGVm7_I_XElUNxo1rSv0n_zL2nmfgsE1n9xDM2_mveYvjbUFwtxBBtvHAH_AIux-ULdXvletKhAwk0CR2cRO0Klnfe3WLUZlDGm2mfXnYx"

In [ ]:
# API constants, shouldn't have to change these.
API_HOST = 'https://api.yelp.com' #The API url header
SEARCH_PATH = '/v3/businesses/search' #The path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # The path to get data for a single business

In [ ]:
def get_color_for_rating(rating):
    if rating >= 4.3:
        return "green"
    elif rating >= 3.5:
        return "orange"
    else:
        return "red"

In [ ]:
def get_restaurants(term,location,number=1):
    #First we get the access token
    #Set up the search data dictionary
    search_data = {
    'term': term,
    'location': location.replace(' ', '+'),
    'limit': number
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % API_KEY,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    return businesses

In [ ]:
while True:
    display(HTML("<h1>Restaurant Search</h1>"))
    display(HTML("Enter a location, or type 'quit' to end this program"))
    location = input()
    
    if location.lower() == 'quit':
        break
    
    display(HTML(f"Enter a cuisine to search for in {location}"))
    term = input()
    
    results = get_restaurants(term,location)
    geodata = geocoder.osm(location)
    foodmap = folium.Map(location = (geodata.lat, geodata.lng), zoom_start = 12)
    
    for rest in results:
        name = rest['name']
        lat = rest['coordinates']['latitude']
        lng = rest['coordinates']['longitude']
        rating = rest['rating']
        phone = rest['phone']
        text = f"{name} rating: {rating} phone number: {phone}"
        marker = folium.Marker(location = (lat, lng), popup = text, icon = folium.Icon(color = get_color_for_rating(rating)))
        foodmap.add_children(marker)
        
    display(foodmap)